In [2]:
import pandas as pd
import gzip
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import numpy as np
from bs4 import BeautifulSoup
from six.moves import html_parser
#from html.parser import HTMLParser
import unicodedata
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
import pickle

/home/ss/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df1 = getDF('baby.json.gz')
df2 = getDF('mus_inst.json.gz')
df3 = getDF('beauty.json.gz')
df4 = getDF('cloth_shoes.json.gz')
df5 = getDF('health_PC.json.gz')
df6 = getDF('home_kitchen.json.gz')
df7 = getDF('off_prod.json.gz')

In [3]:
df1_split = df1[0:1000]
df2_split = df2[0:1000]
df3_split = df3[0:1000]
df4_split = df4[0:1000]
df5_split = df5[0:1000]
df6_split = df6[0:1000]
df7_split = df7[0:1000]

In [4]:
frames = [df1_split, df2_split, df3_split, df4_split, df5_split, df6_split, df7_split]
df = pd.concat(frames)

# categories

In [5]:
columns = ['categories','title']
df_final = pd.DataFrame(df, columns=columns)

In [6]:
def  concatenate_path(category):
     try:
        return ( ' -> '.join((category)[0]))
     except IndexError:
        return 'no-category'
     except TypeError:
        return 'no-category'

In [7]:
df_final['categories_path'] =  df_final['categories'].apply(concatenate_path)
df_final.dropna(subset=['title'], inplace=True)
df_final = df_final[df_final['categories_path'] != 'no-category']

In [8]:
df_final = df_final.reset_index(drop=True)

In [9]:
category_path_df = df_final.groupby('categories_path').agg({'title': 'count'}).sort_values(by='title', ascending=False).reset_index()

In [10]:
category_path_df.sort_values(by='categories_path', inplace=True)
category_path_df['category_path_next'] = category_path_df['categories_path'].shift(-1)

In [11]:
category_path_list = []
for i, value in category_path_df.iterrows():
    category_path = value['categories_path']
    category_path_next = value['category_path_next']
    if str(category_path) not in str(category_path_next):
        category_path_list.append(category_path)

In [12]:
category_path_df = pd.DataFrame(category_path_list, columns=['category_path'])

In [13]:
df_final = df_final[df_final['categories_path'].isin(category_path_df['category_path'])]

In [14]:
def enc_title(cat_title):
    try:
        en_title = unicodedata.normalize('NFKD', unicode(cat_title, 'utf-8', 'ignore')).encode('ascii', 'ignore')
        en_title = BeautifulSoup(en_title, "lxml").encode('ascii', 'ignore')
        en_title = en_title.lower()
        excluded='-/.%'
        en_title=re.split("[^" + excluded + "\w]+",en_title)
        word_set= [token for token in  en_title if not token.isdigit()]
        stop = set(stopwords.words('english'))
        word_set = [w for w in word_set if not w in stop]
        word_set = [w for w in word_set if len(w) > 2]
        en_title=(" ".join(word_set))
        
    except TypeError:  
        en = 'NA'

    return en_title

In [15]:
df_final['title'] =  df_final['title'].apply(enc_title)

In [16]:
df_final = df_final.reset_index(drop=True)

In [17]:
#tr1_x, ts1_x, tr1_y, ts1_y = train_test_split(df_final.title, df_final.categories_path, test_size = 0.2, random_state = 42)


vectorizer = TfidfVectorizer(norm = 'l2',min_df = 0, use_idf = True, smooth_idf = False, sublinear_tf = True,ngram_range=(1,2))
train_vect = vectorizer.fit_transform(df_final.title)
train_vect = train_vect.toarray()
print train_vect.shape

(4567, 27349)


In [18]:
pickle.dump(vectorizer, open("train_vectorizer.pkl", "wb"))
pickle.dump(train_vect, open("train_matrix.pkl", "wb"))
pickle.dump(df_final.categories_path, open("train_result.pkl", "wb"))

In [ ]:
model = pickle.load(open("train_vectorizer.pkl", "rb"))
vect =  pickle.load(open("train_matrix.pkl", "rb"))
lab =  pickle.load(open("train_result.pkl", "rb"))

In [20]:
def FindNeighbor(inp_csr, K):
    output = []
    for row in inp_csr:
        #get single row from input scr matrix
        row = np.array(row)
        #sort and get 'k' nearest neighbor indexes
        max_row = np.argpartition(-row, K)
        top_rows = max_row[:K]
        print max_row
        result = []
        for i in range(len(top_rows)):
            
            val = top_rows[i]
            result.append(lab[val])
            
        output.append(result)
        
    return output

In [3]:

test_vect = model.transform(ts_y)
test_vect = test_vect.toarray()
csr_sim = cosine_similarity(test_vect, vect)
output = FindNeighbor(csr_sim,1)

NameError: name 'model' is not defined

In [22]:
lab[3515]

'Home & Kitchen -> Kitchen & Dining -> Kitchen Knives & Cutlery Accessories -> Carving Knives & Forks -> Carving Knives'